In [6]:
# 🧠 Paso 1: Importar librerías necesarias
from IPython.display import display, Markdown
from IPython.display import Markdown, display
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
import json


# 🔑 Paso 2: Configurar el modelo
model = ChatOpenAI(model="gpt-4o-mini")  # Asegúrate de tener OPENAI_API_KEY exportada

In [7]:
# 🔗 Paso 3: Crear el cliente y conectar al servidor MCP vía SSE

async def crear_agente():
    client = MultiServerMCPClient({
        "transparencia": {
            "url": "http://localhost:8000/sse",
            "transport": "sse",
        }
    })
    await client.__aenter__()
    tools = client.get_tools()
    agent = create_react_agent(model, tools)
    return client, agent

client, agent = await crear_agente()

In [8]:
# 🧾 Paso 4: Realizar consultas de ejemplo

async def preguntar(pregunta: str):
    respuesta = await agent.ainvoke({"messages": pregunta})
    display(Markdown(f"**Pregunta:** {pregunta}\n\n**Respuesta:** {respuesta}"))

In [14]:
def mostrar_respuesta_struct(respuesta):
    """
    Muestra los mensajes estructurados y las herramientas llamadas por el agente.
    """
    if not isinstance(respuesta, dict) or "messages" not in respuesta:
        display(Markdown("⚠️ **Respuesta no válida o sin mensajes.**"))
        return

    mensajes = respuesta["messages"]

    for m in mensajes:
        if isinstance(m, HumanMessage):
            display(Markdown(f"**👤 Usuario:** {m.content}"))

        elif isinstance(m, AIMessage):
            # Mostrar tool_calls si existen
            tool_calls = m.additional_kwargs.get("tool_calls", [])
            for call in tool_calls:
                tool_name = call.get("function", {}).get("name")
                args_str = call.get("function", {}).get("arguments")
                try:
                    args = json.loads(args_str)
                    args_fmt = json.dumps(args, indent=2, ensure_ascii=False)
                except Exception:
                    args_fmt = args_str
                display(Markdown(f"**🧩 Tool Call:** `{tool_name}` con argumentos:\n```json\n{args_fmt}\n```"))

            # Mostrar contenido del modelo (si no está vacío)
            if m.content.strip():
                display(Markdown(f"**🤖 Modelo:** {m.content}"))

        elif isinstance(m, ToolMessage):
            try:
                data = json.loads(m.content)
                formatted = json.dumps(data, indent=2, ensure_ascii=False)
                display(Markdown(f"**🛠️ Tool Response (`{m.name}`):**\n```json\n{formatted}\n```"))
            except json.JSONDecodeError:
                display(Markdown(f"**🛠️ Tool Response (`{m.name}`):**\n{m.content}"))

In [15]:
respuesta = await agent.ainvoke({"messages": "¿Cuántos contratos tiene la empresa con RUC 20512345678?"})
mostrar_respuesta_struct(respuesta)

**👤 Usuario:** ¿Cuántos contratos tiene la empresa con RUC 20512345678?

**🧩 Tool Call:** `buscar_contratos_por_ruc` con argumentos:
```json
{
  "ruc": "20512345678"
}
```

**🛠️ Tool Response (`buscar_contratos_por_ruc`):**
```json
{
  "ruc": "20512345678",
  "total_contratos": 42,
  "monto_total": 1250000.5,
  "entidades_top": [
    "MINSA",
    "Gobierno Regional de Lima"
  ]
}
```

**🤖 Modelo:** La empresa con RUC 20512345678 tiene un total de 42 contratos. El monto total de estos contratos asciende a 1,250,000.5 soles. Las entidades principales con las que tiene contratos son el MINSA y el Gobierno Regional de Lima.

In [16]:
respuesta = await agent.ainvoke({"messages": "¿Cuántos contratos tiene la empresa con RUC 20512345678? y con que entidades tiene contratos?"})
mostrar_respuesta_struct(respuesta)

**👤 Usuario:** ¿Cuántos contratos tiene la empresa con RUC 20512345678? y con que entidades tiene contratos?

**🧩 Tool Call:** `buscar_contratos_por_ruc` con argumentos:
```json
{
  "ruc": "20512345678"
}
```

**🛠️ Tool Response (`buscar_contratos_por_ruc`):**
```json
{
  "ruc": "20512345678",
  "total_contratos": 42,
  "monto_total": 1250000.5,
  "entidades_top": [
    "MINSA",
    "Gobierno Regional de Lima"
  ]
}
```

**🤖 Modelo:** La empresa con RUC 20512345678 tiene un total de 42 contratos. Las principales entidades con las que tiene contratos son:

- MINSA
- Gobierno Regional de Lima

El monto total de los contratos asciende a 1,250,000.5 soles.

In [17]:
respuesta = await agent.ainvoke({"messages": "Hola, que acciones puedes realizar?"})
mostrar_respuesta_struct(respuesta)

**👤 Usuario:** Hola, que acciones puedes realizar?

**🤖 Modelo:** Hola, puedo ayudarte con varias tareas relacionadas con la búsqueda de información. Aquí hay algunas acciones que puedo realizar:

1. **Buscar contratos por RUC**: Puedo encontrar información sobre contratos asociados a un número de RUC específico.
2. **Obtener asistencias de congresistas**: Puedo proporcionarte información sobre las asistencias de un congresista en particular.
3. **Buscar votaciones por tema**: Puedo buscar y mostrar información sobre votaciones relacionadas con un tema específico.

Si necesitas alguno de estos servicios o tienes otra consulta, ¡háznoslo saber!

In [18]:
respuesta = await agent.ainvoke({"messages": "Recuerdas mi pregunta anterior?"})
mostrar_respuesta_struct(respuesta)

**👤 Usuario:** Recuerdas mi pregunta anterior?

**🤖 Modelo:** No tengo la capacidad de recordar conversaciones anteriores. Pero estoy aquí para ayudarte. ¿Cuál es tu pregunta?